In [3]:
# Quick-Inspect for GPC overlays (Nature/Science-ready preview)
# Requirements: ipywidgets, numpy, matplotlib
import re, os, numpy as np, matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown, FloatLogSlider, SelectionRangeSlider, Checkbox, HBox, VBox, HTML, IntSlider, Text, Layout
from matplotlib.ticker import LogLocator, NullFormatter, Formatter
from pathlib import Path

# ---------- Parsing & utilities ----------
_num_pat = re.compile(r"[+-]?(?:\d+\.\d*|\.\d+|\d+)(?:[eE][+-]?\d+)?")

def read_three_cols(path):
    xs, ys, cs = [], [], []
    with open(path, "r", errors="ignore") as f:
        for line in f:
            s = line.strip().replace(",", " ").replace(";", " ")
            nums = _num_pat.findall(s)
            if len(nums) >= 2:
                try:
                    x = float(nums[0]); y = float(nums[1])
                    c = float(nums[2]) if len(nums) >= 3 else np.nan
                    if np.isfinite(x) and np.isfinite(y) and x > 0 and y >= 0:
                        xs.append(x); ys.append(y); cs.append(c)
                except Exception:
                    pass
    x = np.asarray(xs, float); y = np.asarray(ys, float); c = np.asarray(cs, float)
    idx = np.argsort(x); x, y, c = x[idx], y[idx], c[idx]
    if not np.isfinite(c).any(): c = None
    return x, y, c

def baseline_min(y):
    y2 = y - np.nanmin(y)
    y2[y2 < 0] = 0.0
    return y2

def baseline_asls(y, lam=1e5, p=1e-3, niter=10):
    # Eilers & Boelens ASLS baseline
    y = np.asarray(y, float)
    L = len(y)
    D = np.diff(np.eye(L), 2)
    DTD = D.T @ D
    w = np.ones(L)
    for _ in range(niter):
        W = np.diag(w)
        z = np.linalg.solve(W + lam * DTD, w * y)
        w = p * (y > z) + (1 - p) * (y <= z)
    yc = y - z
    yc[yc < 0] = 0.0
    return yc

def smooth_gauss(y, sigma_pts=2.0):
    if sigma_pts <= 0: return y
    rad = max(1, int(3*sigma_pts))
    xs = np.arange(-rad, rad+1, dtype=float)
    k = np.exp(-(xs**2)/(2*sigma_pts*sigma_pts))
    k /= k.sum()
    return np.convolve(y, k, mode="same")

def to_mass_fractions_log_domain(M, Y, domain="ln"):  # domain: 'ln' or 'log10'
    M = np.asarray(M, float); Y = np.asarray(Y, float)
    if domain == "log10":
        x = np.log10(M)
    else:
        x = np.log(M)
    # piecewise Δx for integration
    dx = np.empty_like(x)
    dx[1:-1] = (x[2:] - x[:-2]) / 2.0
    dx[0] = x[1] - x[0]
    dx[-1] = x[-1] - x[-2]
    wi = Y * dx
    s = wi.sum()
    if s > 0: wi /= s
    return wi

def moments_from_wi(M, wi):
    wi = np.asarray(wi, float); M = np.asarray(M, float)
    wi = wi / wi.sum()
    Mw = np.sum(wi * M)
    Mn = 1.0 / np.sum(wi / M)
    Đ = Mw / Mn
    return Mn, Mw, Đ

def crop_with_cum_two_sided(M, Y, Cum, lo=0.001, hi=0.99):
    if Cum is None or not np.isfinite(Cum).any():
        return M, Y
    c = Cum.copy().astype(float)
    if np.nanmax(c) > 1.5: c /= 100.0  # auto-scale % -> fraction
    lo = max(0.0, min(1.0, float(lo)))
    hi = max(0.0, min(1.0, float(hi)))
    if lo >= hi: return M, Y
    m = (c >= lo) & (c <= hi)
    if not np.any(m): return M, Y
    i0 = int(np.argmax(m)); i1 = int(len(m) - np.argmax(m[::-1]))
    return M[i0:i1], Y[i0:i1]

class OneTimesLogFormatter(Formatter):
    def __call__(self, val, pos=None):
        if val <= 0: return ""
        exp = int(np.round(np.log10(val)))
        at = np.isclose(val, 10**exp, rtol=0, atol=1e-12*10**exp)
        return r"$1\times10^{%d}$" % exp if at else ""

# ---------- Load files ----------
file_candidates = [
    p for p in sorted(os.listdir(".")) if p.lower().endswith(".txt")
] + ["/mnt/data/1. 5-TTBTO.txt", "/mnt/data/3. 7-TTBTO.txt", "/mnt/data/cycMo(oct)-2•active.txt"]

file_candidates = [p for p in file_candidates if os.path.exists(p)]
files_dd = Dropdown(options=file_candidates, description="File:", layout=Layout(width="60%"))
add_btn = HTML("<b>Use the dropdown to add curves; re-run to clear.</b>")

# State for multiple curves in one overlay
overlay = []

def _add_current(_=None):
    f = files_dd.value
    if f and os.path.exists(f):
        overlay.append(f)
        status.value = f"Added: <code>{f}</code> (total {len(overlay)})"

add_curve = Dropdown(options=["Add selected file"], description="", layout=Layout(width="20%"))
add_curve.observe(lambda ch: _add_current(), names='value')
status = HTML("")

display(HBox([files_dd, add_curve]), status)

# ---------- Controls ----------
sigma = FloatSlider(value=2.0, min=0.0, max=8.0, step=0.1, description="σ (pts)")
baseline_mode = Dropdown(options=[("Min shift","min"), ("ASLS","asls")], value="min", description="Baseline")
asls_lam = FloatLogSlider(value=3e5, base=10, min=3, max=7, step=0.1, description="ASLS λ")
asls_p   = FloatLogSlider(value=1e-3, base=10, min=-4, max=-1, step=0.1, description="ASLS p")
cum_lo = Text(value="0.1%", description="Cum low")
cum_hi = Text(value="99%", description="Cum high")
norm = Dropdown(options=[("W(lnM)","ln"), ("W(log10M)","log10"), ("Raw","none")], value="ln", description="Normalize")
minor_ticks = Checkbox(value=True, description="Minor ticks 2–9")
decades = IntSlider(value=0, min=0, max=10, step=1, description="Limit decades (0=auto)")

controls = VBox([HBox([sigma, baseline_mode]),
                 HBox([asls_lam, asls_p]),
                 HBox([cum_lo, cum_hi, norm]),
                 HBox([minor_ticks, decades])])

display(controls)

# ---------- Plot function ----------
def _to_frac(s):
    s = str(s).strip()
    if s.endswith("%"):
        return max(0.0, min(1.0, float(s[:-1]) / 100.0))
    v = float(s)
    return v/100.0 if v > 1.0 else max(0.0, min(1.0, v))

def quick_plot(_sigma, _baseline_mode, _lam, _p, _cum_lo, _cum_hi, _norm, _minor, _decades):
    if not overlay:
        print("No curves added yet. Use the dropdown above to add files.")
        return
    fig, ax = plt.subplots(figsize=(8,5))
    results = []
    xmin_all, xmax_all = np.inf, 0.0
    for path in overlay:
        M, Y, Cum = read_three_cols(path)
        # two-sided cumulative crop
        try:
            lo = _to_frac(_cum_lo); hi = _to_frac(_cum_hi)
        except Exception:
            lo, hi = 0.0, 1.0
        if lo is not None or hi is not None:
            lo = 0.0 if lo is None else lo
            hi = 1.0 if hi is None else hi
            M, Y = crop_with_cum_two_sided(M, Y, Cum, lo=lo, hi=hi)
        # baseline
        if _baseline_mode == "asls":
            Y = baseline_asls(Y, lam=_lam, p=_p, niter=10)
        else:
            Y = baseline_min(Y)
        # smoothing & clamp
        Y = smooth_gauss(Y, sigma_pts=_sigma)
        Y[Y < 0] = 0.0
        # normalization
        if _norm == "log10":
            A = np.trapz(Y, x=np.log10(M))
            if A > 0: Y = Y / A
        elif _norm == "ln":
            A = np.trapz(Y, x=np.log(M))
            if A > 0: Y = Y / A
        # stats
        Mp = float(M[np.nanargmax(Y)])
        if _norm == "none":
            wi = to_mass_fractions_log_domain(M, Y, domain="ln")  # default domain for stats
        else:
            wi = to_mass_fractions_log_domain(M, Y, domain=_norm)
        Mn, Mw, Đ = moments_from_wi(M, wi)
        label = f"{Path(path).stem}  (Mw={Mw:,.0f}, Đ={Đ:0.2f})"
        ax.plot(M, Y, lw=2, label=label)
        ax.axvline(Mp, ymin=0, ymax=0.95, ls="--", lw=1, alpha=0.7)
        ax.axvline(Mn, ymin=0, ymax=0.95, ls=":", lw=1, alpha=0.7)
        ax.axvline(Mw, ymin=0, ymax=0.95, ls="-.", lw=1, alpha=0.7)
        xmin_all = min(xmin_all, M.min()); xmax_all = max(xmax_all, M.max())
        results.append((Path(path).name, Mp, Mn, Mw, Đ))
    # axes
    ax.set_xscale("log")
    # major decades
    ax.xaxis.set_major_locator(LogLocator(base=10.0))
    ax.xaxis.set_major_formatter(OneTimesLogFormatter())
    # minor ticks
    if _minor:
        ax.xaxis.set_minor_locator(LogLocator(base=10.0, subs=np.arange(2,10)*0.1))
        ax.xaxis.set_minor_formatter(NullFormatter())
    else:
        ax.xaxis.set_minor_locator(LogLocator(base=10.0, subs=[]))
        ax.xaxis.set_minor_formatter(NullFormatter())
    # decades limit
    if _decades and np.isfinite(xmin_all) and xmax_all > 0:
        lo_dec = int(np.floor(np.log10(xmin_all)))
        hi_dec = lo_dec + int(_decades)
        ax.set_xlim(10**lo_dec, 10**hi_dec)
    else:
        ax.set_xlim(max(1.0, xmin_all), xmax_all)
    # labels & grid
    ylab = "W(lnM)" if _norm=="ln" else ("W(log10M)" if _norm=="log10" else "Signal (arb.)")
    ax.set_xlabel(r"Molecular weight, $M$ (g mol$^{-1}$)")
    ax.set_ylabel(ylab)
    ax.set_ylim(bottom=0.0)
    ax.grid(True, which="both", alpha=0.25, linestyle="-", linewidth=0.6)
    ax.legend(loc="upper left", frameon=True, fontsize=9)
    plt.show()

    # summary table
    rows = []
    for name, Mp, Mn, Mw, Đ in results:
        rows.append(f"<tr><td>{name}</td><td>{Mp:,.0f}</td><td>{Mn:,.0f}</td><td>{Mw:,.0f}</td><td>{Đ:0.2f}</td></tr>")
    html = """<table style="border-collapse:collapse">
    <tr><th style="text-align:left;padding-right:12px">Sample</th>
        <th style="text-align:right;padding-right:12px">Mp</th>
        <th style="text-align:right;padding-right:12px">Mn</th>
        <th style="text-align:right;padding-right:12px">Mw</th>
        <th style="text-align:right;padding-right:12px">Đ</th></tr>
    """ + "\n".join(rows) + "</table>"
    display(HTML(html))

interact(
    quick_plot,
    _sigma=sigma,
    _baseline_mode=baseline_mode,
    _lam=asls_lam,
    _p=asls_p,
    _cum_lo=cum_lo,
    _cum_hi=cum_hi,
    _norm=norm,
    _minor=minor_ticks,
    _decades=decades
);


HTML(value='')

interactive(children=(FloatSlider(value=2.0, description='σ (pts)', max=8.0), Dropdown(description='Baseline',…